<a href="https://colab.research.google.com/github/ZainabSMU/GenAI/blob/main/GenAI_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Building the general corpus from BookCorpus

!pip install datasets nltk

from datasets import load_dataset
from nltk.tokenize import sent_tokenize
import nltk, os, random

nltk.download("punkt")
nltk.download("punkt_tab")


os.makedirs("data", exist_ok=True)

print("Loading BookCorpus from community dataset...")
book = load_dataset("rojagtap/bookcorpus", split="train")

sentences = []
max_sentences = 200_000  # adjust if needed

print("Processing BookCorpus...")

for row in book:
    text = row["text"]
    if not text or text.strip() == "":
        continue

    # Sentence split
    sents = sent_tokenize(text)
    for s in sents:
        s = s.strip()
        if len(s.split()) > 5:  # filter out too-short sentences
            sentences.append(s)

    if len(sentences) >= max_sentences:
        break

print(f"Collected {len(sentences)} sentences.")

random.shuffle(sentences)

out_path = "data/general_corpus.txt"
with open(out_path, "w", encoding="utf-8") as f:
    for s in sentences:
        f.write(s + "\n")

print("Saved general corpus to:", out_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading BookCorpus from community dataset...


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

books_large_p1.txt:   0%|          | 0.00/2.52G [00:00<?, ?B/s]

books_large_p2.txt:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

Processing BookCorpus...
Collected 200000 sentences.
Saved general corpus to: data/general_corpus.txt


In [ ]:
!pip install nltk beautifulsoup4

import os, requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk

# Download both punkt + punkt_tab for sentence tokenization
nltk.download("punkt")
nltk.download("punkt_tab")

os.makedirs("data", exist_ok=True)

BASE = "http://www.paulgraham.com/"
index_url = BASE + "articles.html"

print("Fetching essay index:", index_url)
index_html = requests.get(index_url).text
index_soup = BeautifulSoup(index_html, "html.parser")

# Collect all internal .html links (the essays)
links = []
for a in index_soup.find_all("a", href=True):
    href = a["href"]
    if href.endswith(".html") and not href.startswith("http"):
        links.append(href)

# Deduplicate while preserving order
links = list(dict.fromkeys(links))
print(f"Found {len(links)} essay links.")

sentences = []

for href in links:
    url = BASE + href
    print("Scraping:", url)
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")

    # Main essay text is in <p> tags
    for p in soup.find_all("p"):
        text = p.get_text(" ", strip=True)
        if not text:
            continue
        for s in sent_tokenize(text):
            s = s.strip()
            # filter super-short/noisy lines
            if len(s.split()) > 4:
                sentences.append(s)

print(f"Total PG sentences collected: {len(sentences)}")

out_path = "data/pg_corpus.txt"
with open(out_path, "w", encoding="utf-8") as f:
    for s in sentences:
        f.write(s + "\n")

print("Saved PG corpus to:", out_path)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Fetching essay index: http://www.paulgraham.com/articles.html
Found 230 essay links.
Scraping: http://www.paulgraham.com/index.html
Scraping: http://www.paulgraham.com/greatwork.html
Scraping: http://www.paulgraham.com/kids.html
Scraping: http://www.paulgraham.com/selfindulgence.html
Scraping: http://www.paulgraham.com/field.html
Scraping: http://www.paulgraham.com/goodwriting.html
Scraping: http://www.paulgraham.com/do.html
Scraping: http://www.paulgraham.com/woke.html
Scraping: http://www.paulgraham.com/writes.html
Scraping: http://www.paulgraham.com/when.html
Scraping: http://www.paulgraham.com/foundermode.html
Scraping: http://www.paulgraham.com/persistence.html
Scraping: http://www.paulgraham.com/reddits.html
Scraping: http://www.paulgraham.com/google.html
Scraping: http://www.paulgraham.com/best.html
Scraping: http://www.paulgraham.com/superlinear.html
Scraping: http://www.paulgraham.com/getideas.html
Scraping: http://www.paulgraham.com/read.html
Scraping: http://www.paulgraham.c

In [ ]:
# Doing a quick sanity check for both the corpus' above
!head -n 5 data/pg_corpus.txt
!head -n 5 data/general_corpus.txt

A wealth tax will usually have a threshold at which it starts.
How much difference would a high threshold make?
To model that,
we need to make some assumptions about the initial value of
your stock and the growth rate.
it was said that in the minaret of the al-karaouine mosque there was a special room , the dar al-muwaqqit , where the times of daily prayer were established for the city .
something about the crystal was wiping his mind clean , stripping him of his memories and emotions , leaving him an empty vessel .
without thinking she stepped back , almost tripping over trunk 's large feet .
this young man had risked his life in an attempt to rescue sokolov 's companions , and he had come to the conclusion that jake was someone he could trust to do his best to help in any circumstance .
im cleora , she said as she offered her hand .


In [ ]:
# part1_simcse_train.py

import os
import json
from dataclasses import dataclass
from typing import List

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import (
    BertTokenizerFast,
    BertModel,
    get_cosine_schedule_with_warmup,
)
from torch.optim import AdamW


# =========================
#  Config
# =========================

@dataclass
class TrainingConfig:
    model_name: str
    train_file: str
    output_dir: str
    max_length: int = 128
    batch_size: int = 256
    lr: float = 2e-5
    weight_decay: float = 0.01
    num_epochs: int = 1
    warmup_ratio: float = 0.1
    temperature: float = 0.05
    eval_every_steps: int = 0  # set >0 if you later add eval
    device: str = "cuda" if torch.cuda.is_available() else "cpu"


# =========================
#  Dataset
# =========================

class TextLineDataset(Dataset):
    """
    Expects a plain text file with ONE sentence per line.
    Blank lines are ignored.
    """
    def __init__(self, path: str):
        if not os.path.exists(path):
            raise FileNotFoundError(f"Train file not found: {path}")
        with open(path, "r", encoding="utf-8") as f:
            self.lines = [line.strip() for line in f if line.strip()]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        return self.lines[idx]


def collate_fn(batch: List[str], tokenizer: BertTokenizerFast, max_length: int):
    """
    Tokenizes a batch of sentences.
    """
    enc = tokenizer(
        batch,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )
    return enc


# =========================
#  SimCSE Model
# =========================

class SimCSEModel(nn.Module):
    """
    Simple SimCSE-style model:
      - BERT encoder
      - CLS pooling
      - Tanh projection
      - InfoNCE contrastive loss with in-batch negatives
    """
    def __init__(self, model_name: str, temperature: float = 0.05):
        super().__init__()
        # NOTE: This loads a pretrained BERT. If you truly want "random init
        # from scratch", replace this with:
        #   from transformers import BertConfig
        #   config = BertConfig()
        #   self.encoder = BertModel(config)
        self.encoder = BertModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        self.projection = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
        )
        self.temperature = temperature

    def encode(self, batch):
        """
        Returns normalized sentence embeddings.
        """
        outputs = self.encoder(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
        )
        cls = outputs.last_hidden_state[:, 0]  # [CLS] token
        proj = self.projection(cls)
        emb = nn.functional.normalize(proj, p=2, dim=-1)
        return emb

    def forward(self, batch):
        """
        SimCSE unsupervised:
          - Run encoder twice with dropout
          - Positive pairs = (z1_i, z2_i)
          - Negatives = others in batch
        """
        z1 = self.encode(batch)
        z2 = self.encode(batch)

        batch_size = z1.size(0)
        # Similarity matrix [B, B]
        sim = torch.matmul(z1, z2.t()) / self.temperature

        labels = torch.arange(batch_size, device=z1.device)
        loss = nn.CrossEntropyLoss()(sim, labels)
        return loss


# =========================
#  Training loop
# =========================

def train_simcse(config: TrainingConfig):
    os.makedirs(config.output_dir, exist_ok=True)

    print("Loading tokenizer and dataset...")
    tokenizer = BertTokenizerFast.from_pretrained(config.model_name)
    train_dataset = TextLineDataset(config.train_file)
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        drop_last=True,
        collate_fn=lambda batch: collate_fn(batch, tokenizer, config.max_length),
    )

    print(f"Dataset size: {len(train_dataset)} lines")
    print("Building model...")
    model = SimCSEModel(config.model_name, temperature=config.temperature)
    model.to(config.device)

    optimizer = AdamW(
        model.parameters(),
        lr=config.lr,
        weight_decay=config.weight_decay,
    )

    total_steps = len(train_loader) * config.num_epochs
    warmup_steps = int(total_steps * config.warmup_ratio)
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps,
    )

    global_step = 0
    loss_log = []

    print("Starting training...")
    model.train()
    for epoch in range(config.num_epochs):
        for batch in train_loader:
            batch = {k: v.to(config.device) for k, v in batch.items()}

            loss = model(batch)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            global_step += 1
            loss_val = float(loss.item())
            loss_log.append(loss_val)

            if global_step % 100 == 0:
                print(f"Epoch {epoch+1} | Step {global_step} | Loss {loss_val:.4f}")

    # Save final model and loss log
    final_model_path = os.path.join(config.output_dir, "final_model.pt")
    torch.save(model.state_dict(), final_model_path)

    loss_log_path = os.path.join(config.output_dir, "loss_log.json")
    with open(loss_log_path, "w", encoding="utf-8") as f:
        json.dump(loss_log, f)

    print(f"Training complete. Model saved to: {final_model_path}")
    print(f"Loss log saved to: {loss_log_path}")


# =========================
#  Main
# =========================

if __name__ == "__main__":
    # -------- General-purpose embedding model --------
    general_cfg = TrainingConfig(
        model_name="bert-base-uncased",
        train_file="data/general_corpus.txt",      # <-- change path
        output_dir="checkpoints/general_simcse",
        max_length=128,
        batch_size=128,   # 256 -> 128 or even 64
        num_epochs=1,
        lr=2e-5,
    )
    train_simcse(general_cfg)

    # -------- Paul Graham–specific embedding model --------
    pg_cfg = TrainingConfig(
        model_name="bert-base-uncased",
        train_file="data/pg_corpus.txt",          # <-- change path
        output_dir="checkpoints/pg_simcse",
        max_length=256,
        batch_size=64,    # 128 -> 64 or 32
        num_epochs=3,
        lr=2e-5,
    )
    train_simcse(pg_cfg)

Loading tokenizer and dataset...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Dataset size: 200000 lines
Building model...


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Starting training...
Epoch 1 | Step 100 | Loss 0.0014
Epoch 1 | Step 200 | Loss 0.0005
Epoch 1 | Step 300 | Loss 0.0004
Epoch 1 | Step 400 | Loss 0.0003
Epoch 1 | Step 500 | Loss 0.0002
Epoch 1 | Step 600 | Loss 0.0002
Epoch 1 | Step 700 | Loss 0.0003
Epoch 1 | Step 800 | Loss 0.0001
Epoch 1 | Step 900 | Loss 0.0002
Epoch 1 | Step 1000 | Loss 0.0001
Epoch 1 | Step 1100 | Loss 0.0001
Epoch 1 | Step 1200 | Loss 0.0001
Epoch 1 | Step 1300 | Loss 0.0002
Epoch 1 | Step 1400 | Loss 0.0001
Epoch 1 | Step 1500 | Loss 0.0001
Training complete. Model saved to: checkpoints/general_simcse/final_model.pt
Loss log saved to: checkpoints/general_simcse/loss_log.json
Loading tokenizer and dataset...
Dataset size: 28570 lines
Building model...
Starting training...
Epoch 1 | Step 100 | Loss 0.0036
Epoch 1 | Step 200 | Loss 0.0009
Epoch 1 | Step 300 | Loss 0.0210
Epoch 1 | Step 400 | Loss 0.0014
Epoch 2 | Step 500 | Loss 0.0088
Epoch 2 | Step 600 | Loss 0.0004
Epoch 2 | Step 700 | Loss 0.0003
Epoch 2 | St

In [ ]:
# part1_plot_loss.py

import json
import os
import matplotlib.pyplot as plt

os.makedirs("plots", exist_ok=True)

def load_loss(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def plot_loss(loss_values, title: str, out_path: str):
    plt.figure()
    plt.plot(loss_values)
    plt.title(title)
    plt.xlabel("Training step")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.savefig(out_path, dpi=300)
    plt.close()
    print(f"Saved plot to {out_path}")


if __name__ == "__main__":
    # General-purpose model loss
    gen_loss_path = "checkpoints/general_simcse/loss_log.json"
    gen_loss = load_loss(gen_loss_path)
    plot_loss(
        gen_loss,
        title="General-purpose SimCSE Training Loss",
        out_path="plots/general_loss.png",
    )

    # Paul Graham model loss
    pg_loss_path = "checkpoints/pg_simcse/loss_log.json"
    if os.path.exists(pg_loss_path):
        pg_loss = load_loss(pg_loss_path)
        plot_loss(
            pg_loss,
            title="Paul Graham SimCSE Training Loss",
            out_path="plots/pg_loss.png",
        )


In [ ]:
# part1_eval_retrieval.py

import json
import os
from typing import List

import torch
import numpy as np
from transformers import BertTokenizerFast

from part1_simcse_train import SimCSEModel  # same class as before


def encode_texts(
    model: SimCSEModel,
    tokenizer: BertTokenizerFast,
    texts: List[str],
    device: str,
    batch_size: int = 64,
    max_length: int = 256,
):
    model.eval()
    all_embs = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i : i + batch_size]
            enc = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt",
            )
            enc = {k: v.to(device) for k, v in enc.items()}
            embs = model.encode(enc)  # uses encode() method
            all_embs.append(embs.cpu())
    return torch.cat(all_embs, dim=0).numpy()


def mrr_at_k(rank_lists: List[List[int]], ground_truth: List[List[int]], k: int = 10):
    mrr = 0.0
    for ranked, gt in zip(rank_lists, ground_truth):
        rank = None
        for i, doc_id in enumerate(ranked[:k]):
            if doc_id in gt:
                rank = i + 1
                break
        if rank is not None:
            mrr += 1.0 / rank
    return mrr / len(rank_lists)


def ndcg_at_k(rank_lists: List[List[int]], ground_truth: List[List[int]], k: int = 10):
    def dcg(ranked, gt, k):
        score = 0.0
        for i, doc_id in enumerate(ranked[:k]):
            if doc_id in gt:
                score += 1.0 / np.log2(i + 2)  # i+2 so first rank has log2(2)=1
        return score

    ndcgs = []
    for ranked, gt in zip(rank_lists, ground_truth):
        ideal = dcg(gt, gt, min(k, len(gt))) if gt else 0.0
        actual = dcg(ranked, gt, k)
        ndcgs.append(0.0 if ideal == 0.0 else actual / ideal)
    return float(np.mean(ndcgs))


if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # ---------- Load model ----------
    model_name = "bert-base-uncased"
    tokenizer = BertTokenizerFast.from_pretrained(model_name)

    model = SimCSEModel(model_name=model_name, temperature=0.05)
    ckpt_path = "checkpoints/pg_simcse/final_model.pt"  # for example
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(ckpt_path)
    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    model.to(device)

    # ---------- Load corpus ----------
    with open("data/val_corpus.txt", "r", encoding="utf-8") as f:
        corpus = [line.strip() for line in f if line.strip()]

    corpus_embs = encode_texts(model, tokenizer, corpus, device=device)

    # ---------- Load queries ----------
    queries = []
    rel_ids = []
    with open("data/val_queries.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            queries.append(obj["query"])
            rel_ids.append(obj["relevant_ids"])

    query_embs = encode_texts(model, tokenizer, queries, device=device)

    # ---------- Compute ranking ----------
    rank_lists = []
    for q_vec in query_embs:
        # cosine similarity
        sims = corpus_embs @ q_vec
        ranked_ids = np.argsort(-sims)  # descending
        rank_lists.append(list(ranked_ids))

    mrr = mrr_at_k(rank_lists, rel_ids, k=10)
    ndcg = ndcg_at_k(rank_lists, rel_ids, k=10)

    print(f"MRR@10:  {mrr:.4f}")
    print(f"nDCG@10: {ndcg:.4f}")
